<a href="https://colab.research.google.com/github/nahbos/AUT-Advanced-Information-Retrieval/blob/main/Ex02/neural_network_based_information_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sobhan Moradian Daghigh

- 12-13-2022

### Ex-02: NN-based IR

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 6.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import scipy
import re
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
from collections import Counter
import random
import pickle
from tqdm import tqdm
from PIL import Image


import transformers
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, Trainer
from transformers import TrainingArguments
from transformers import DataCollatorWithPadding

In [3]:
!wget -nc https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/train_data.csv
!wget -nc https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/valid_data.csv
!wget -nc https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/test_data.csv
!wget -nc https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/bigram_LM.png
!wget -nc https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/diricklet_smoothing_LM.png

--2023-01-19 17:36:23--  https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/train_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5148485 (4.9M) [text/plain]
Saving to: ‘train_data.csv’

train_data.csv      100%[===================>]   4.91M  --.-KB/s    in 0.02s   

2023-01-19 17:36:23 (274 MB/s) - ‘train_data.csv’ saved [5148485/5148485]

--2023-01-19 17:36:24--  https://raw.githubusercontent.com/nahbos/Advanced-Information-Retrieval/main/Ex01/Data/valid_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting 

# Part One. 
* Data Loading

In [4]:
train = pd.read_csv('./train_data.csv')
val   = pd.read_csv('./valid_data.csv')
test  = pd.read_csv('./test_data.csv')

In [5]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,14,29,30,What are the laws to change your status from a...,What are the laws to change your status from a...,0
1,18,37,38,Why are so many Quora users posting questions ...,Why do people ask Quora questions which can be...,1
2,38,77,78,How do we prepare for UPSC?,How do I prepare for civil service?,1
3,58,117,118,I was suddenly logged off Gmail. I can't remem...,I can't remember my Gmail password or my recov...,1
4,60,121,122,How do I download content from a kickass torre...,Is Kickass Torrents trustworthy?,0


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37250 entries, 0 to 37249
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            37250 non-null  int64 
 1   qid1          37250 non-null  int64 
 2   qid2          37250 non-null  int64 
 3   question1     37250 non-null  object
 4   question2     37250 non-null  object
 5   is_duplicate  37250 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 1.7+ MB


In [7]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1010 non-null   int64 
 1   qid1          1010 non-null   int64 
 2   qid2          1010 non-null   int64 
 3   question1     1010 non-null   object
 4   question2     1010 non-null   object
 5   is_duplicate  1010 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.5+ KB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            980 non-null    int64 
 1   qid1          980 non-null    int64 
 2   qid2          980 non-null    int64 
 3   question1     980 non-null    object
 4   question2     980 non-null    object
 5   is_duplicate  980 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 46.1+ KB


## Preprocess

In [9]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953

    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q

In [10]:
train_org = train.copy()
train['question1'] = train['question1'].apply(preprocess)
train['question2'] = train['question2'].apply(preprocess)

val_org = val.copy()
val['question1'] = val['question1'].apply(preprocess)
val['question2'] = val['question2'].apply(preprocess)

test_org = test.copy()
test['question1'] = test['question1'].apply(preprocess)
test['question2'] = test['question2'].apply(preprocess)

**Ok, Everything looks right ))**

# Part Two.
* Evaluation Metrics

### Average Precision (AP)

In [11]:
def eval_AP(ref, pred, k):
    precision = 0
    for doc_id in ref:
        ref1, k_pred = ref[doc_id], pred[doc_id][:k]
        precision += (eval_p_at_k(ref1, k_pred) / len(ref))
    return precision

In [12]:
import copy 
def eval_MAP(ref, pred):
    mean_average_precision = 0
    for doc_id in ref:
        sum_avg_precision = 0
        ref1, pred1 = ref[doc_id], pred[doc_id]
        ref_unseen = copy.deepcopy(ref1)
        for k in range(1, 11):
            k_pred = pred1[:k]
            if k_pred[-1] in ref_unseen:
                ref_unseen.remove(k_pred[-1])
                sum_avg_precision += (eval_p_at_k(ref1, k_pred) / len(ref1))
        mean_average_precision += (sum_avg_precision / len(ref))
    return mean_average_precision

### Reciprocal Rank (RR)

In [13]:
def eval_RR(ref, pred):
    for i, p in enumerate(pred):
        if p in ref:
            return 1 / (i+1)
    return 0

In [14]:
def eval_MRR(ref, pred):
    mean_reciprocal_rank = 0
    for doc_id in ref:
        mean_reciprocal_rank += eval_RR(ref[doc_id], pred[doc_id]) / len(ref)
    return mean_reciprocal_rank

### Precision @ k

In [15]:
def eval_p_at_k(ref, k_pred):
    pred_relevent = len(set(k_pred).intersection(set(ref)))
    return pred_relevent / len(k_pred)

# Part Three.
Bert

In [16]:
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

In [17]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
with torch.no_grad():
    model.eval()
    model.to('cuda')
    doc_repr = dict()
    for _, (doc_id, doc) in tqdm(train[['qid2', 'question2']].iterrows(), total=len(train)):
        inputs = tokenizer(doc, return_tensors="pt").to('cuda')
        outputs = model(**inputs)
        doc_repr[doc_id] = outputs.last_hidden_state.mean(dim=1).squeeze()

100%|██████████| 37250/37250 [06:55<00:00, 89.75it/s]


In [19]:
with torch.no_grad():
    model.eval()
    model.to('cuda')
    query_answer_bert = dict()
    for _, (qid, qtext) in tqdm(test[['qid1', 'question1']].iterrows(), total=len(test)):
        if qid in query_answer_bert:
            continue
        inputs = tokenizer(qtext, return_tensors="pt").to('cuda')
        output = model(**inputs)
        query_vec = output.last_hidden_state.mean(dim=1).squeeze()
        doc_score = []
        for doc_id in doc_repr:
            doc_vec = doc_repr[doc_id]
            sim = F.cosine_similarity(query_vec, doc_vec, dim=0).item()
            doc_score.append((doc_id, sim))
        doc_score = pd.DataFrame(doc_score, columns =['id', 'score']).sort_values('score', ascending=False)
        topk = doc_score.nlargest(n=10, columns='score')
        query_answer_bert[qid] = topk['id'].tolist()

100%|██████████| 980/980 [04:09<00:00,  3.93it/s]


In [20]:
query_answer_ref = dict()
test_qid1_grouped = test.groupby('qid1')
for qid1 in test.qid1.unique():
    group = test_qid1_grouped.get_group(qid1)
    query_answer_ref[qid1] = group['qid2'].values.tolist()

In [21]:
map_ = eval_MAP(ref=query_answer_ref, pred=query_answer_bert)
mrr = eval_MRR(ref=query_answer_ref, pred=query_answer_bert)
avg_p_at_5 = eval_AP(ref=query_answer_ref, pred=query_answer_bert, k=5)
avg_p_at_10 = eval_AP(ref=query_answer_ref, pred=query_answer_bert, k=10)

print('MAP : {:.3f}'.format(map_))
print('MRR : {:.3f}'.format(mrr))
print('P@5 : {:.3f}'.format(avg_p_at_5))
print('P@10: {:.3f}'.format(avg_p_at_10))

MAP : 0.453
MRR : 0.777
P@5 : 0.484
P@10: 0.339


# Part Four.

# Finito